In [1]:
from action_dataloader import BatchGenerator, parse_actions_dict
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Required parameters
mapping_file = "/home/user/visisonrd-action-segmentation/visisonrd-action-segmentation/data/mapping.txt"  # Replace with the actual path to your mapping.txt
gt_path = "path/to/groundTruth"       # Replace with the actual path to ground truth files
features_path = "path/to/features"    # Replace with the actual path to feature files
model_type = "mstcn"                  # "mstcn" or "asformer"
sample_rate = 1                       # Adjust as needed
batch_size = 4
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parse the action mapping file
actions_dict = parse_actions_dict(mapping_file)

# Get the number of classes from the action mapping
num_classes = len(actions_dict)

# Initialize the BatchGenerator
batch_gen = BatchGenerator(num_classes, actions_dict, gt_path, features_path, sample_rate, model_type)

In [ ]:
batch_gen.load_examples()

In [ ]:
batch_size = 4  # Number of samples per batch
while batch_gen.has_next():
    batch_input, batch_target, mask, batch_videos = batch_gen.next_batch(batch_size, if_warp=(model_type == "asformer"))
    print(f"Batch Input Shape: {batch_input.shape}, Target Shape: {batch_target.shape}, Mask Shape: {mask.shape}")

In [ ]:
# Define a simple model
model = nn.Sequential(
    nn.Conv1d(in_channels=2048, out_channels=512, kernel_size=3, padding=1),  # Example input feature dim = 2048
    nn.ReLU(),
    nn.Conv1d(in_channels=512, out_channels=num_classes, kernel_size=1)
)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=-100)  # Ignore -100 in targets
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
for epoch in range(num_epochs):
    batch_gen.reset()  # Reset and shuffle at the start of each epoch
    while batch_gen.has_next():
        batch_input, batch_target, mask, batch_videos = batch_gen.next_batch(batch_size, if_warp=(model_type == "asformer"))
        
        # Move tensors to the correct device (e.g., GPU)
        batch_input = batch_input.to(device)
        batch_target = batch_target.to(device)
        
        # Forward pass through the model
        outputs = model(batch_input)
        
        # Compute loss (example using CrossEntropyLoss)
        loss = criterion(outputs, batch_target)
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()